In [314]:
"""
This file implements a LSTM for the image manipulation-eye gaze timeseries data set.
It is designed to solve the classification problem about predicting if a picture is manipulated based on a sequence of 
data on their eye gaze.
"""

import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

In [315]:
"""
Step 1: load data
"""
# load all data
data = pd.read_excel('Caldwell_ImageManipulation-EyeGaze_DataSetCombined.xlsx',
                        sheet_name='data')

data = data[["participant", "image", "image manipulated", "vote"]]

data_extended = pd.read_csv('Caldwell_Manip_Images_10-14_TimeSeries.csv')
# rename columns to make them align
data_extended = data_extended.rename(index=str, columns={'Participant_ID': 'participant', 'Image_ID': 'image'})
data = pd.merge(data_extended, data, how="left", on=["participant", "image"])    # join the dataframes
data = data.sort_values(by=["Start Time"])

# Min-Max scaling normalization
for column in range(data.shape[1] - 2):
    temp = data.iloc[:, column]  
    ma = temp.max()
    mi = temp.min()
    data.iloc[:, column] = data.iloc[:, column].apply(lambda x: (x - mi) / (ma - mi))

# separate the data into training set(image 10-12), validation set(image 13), test set(image 14). 
# 13 is 3/4 = 0.75 after normalization
traindata = data.loc[data['image'] < 0.75]
validatedata = data.loc[data['image'] == 0.75]
testdata = data.loc[data['image'] == 1.0]

# separate the data into input and target
train_input = traindata.iloc[:, :9]
train_target = traindata.iloc[:, 9]

validate_input = validatedata.iloc[:, :9]
validate_target = validatedata.iloc[:, 9]

test_input = testdata.iloc[:, :9]
test_target = testdata.iloc[:, 9]

X_train = Variable(torch.Tensor(train_input.values).float())
Y_train = Variable(torch.Tensor(train_target.values).long())

X_validate = Variable(torch.Tensor(validate_input.values).float())
Y_validate = Variable(torch.Tensor(validate_target.values).long())

X_test = Variable(torch.Tensor(test_input.values).float())
Y_test = Variable(torch.Tensor(test_target.values).long())
X_train.shape

torch.Size([19002, 9])

In [316]:
"""
Step 2: Data preprocessing
"""
# hyperparameters
input_dim = 9   # no. of input features
output_dim = 2  # no. of output classes
hidden_dim = 6  # no. of units in hidden state
num_layers = 2  # number of LSTM layers
batch = 32      # batch size
seq_length = 10 # sequence length
learning_rate = 0.01
num_epochs = 200
    
#use minibatch to preprocess data
class PrepareData(Dataset):

    def __init__(self, X, y):
        if not torch.is_tensor(X):
            self.X = torch.from_numpy(X)
        if not torch.is_tensor(y):
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# create batches with size = batch_size*seq_length, build sequences later
train_batchs = PrepareData(X=np.array(train_input), y=np.array(train_target))
train_batchs = DataLoader(train_batchs, batch_size=batch * seq_length, shuffle=False)

In [317]:
"""
Step 3: Define the LSTM model
"""
class LSTM(nn.Module):
    """reference https://github.com/jessicayung/blog-code-snippets/blob/master/lstm-pytorch/lstm-baseline.py"""
    def __init__(self, input_dim, hidden_dim, batch, seq_len, output_dim, num_layers):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch = batch
        self.num_layers = num_layers
        self.seq_len = seq_length
    
        # the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first = True)
        
        # the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)
        
    def init_hidden(self):
        """This is called each time a sequence is fully learned, then the hidden state has to be reinitialized"""
        return (torch.zeros(self.num_layers, self.batch, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch, self.hidden_dim))
    
    def forward(self, input):
        """Forward pass through LSTM layer"""
        # shape of self.hidden: (a, b), where a and b both have shape (num_layers, batch_size, hidden_dim).
        # input has size batch * seq length * input_dim
        lstm_out, self.hidden = self.lstm(input.view(self.batch, self.seq_len, -1))
        
        # Only take the output from the final timetep
        y_pred = self.linear(lstm_out[:, -1, :]) 
        return y_pred

In [318]:
"""
Step 4: Train the model
"""

model = LSTM(input_dim= input_dim, hidden_dim= hidden_dim, batch= batch, seq_len= seq_length, output_dim= output_dim, num_layers= num_layers)

loss_f = nn.CrossEntropyLoss()  # classification

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)   # minibatch gradient descent

losses = np.array([])
accuracies = np.array([])
previous_two_validation_error = [float("inf"), float("inf")]

"""Train and validate the model"""
for e in range(num_epochs):
    
    losses = np.array([])        # reinitialize each time to calculate average loss
    accuracies = np.array([])    # reinitialize each time to calculate average accuracy
    
    model.batch = batch          # restate batch size as it may has been changed by validation
    model.seq_len = seq_length   # restate sequence length

    counter = 0                  # count the number of batchs
     
    # idea from https://conorsdatablog.wordpress.com/2018/05/03/up-and-running-with-pytorch-minibatching-dataloading-and-model-building/
    for ix, (x, y) in enumerate(train_batchs):
        # only learn from full batchs(size = batchsize *seqlength)
        if x.shape[0] < batch * seq_length:     
            continue
        
        counter += 1
        model.hidden = model.init_hidden()    # reinitialize hidden state each sequence
        
        y = y.view(batch, seq_length)[:,-1]   # resize target into sequence
        
        _X = Variable(x).float().view(batch, seq_length, -1)
        _Y = Variable(y).long()

        y_pred = model(_X)
        loss = loss_f(y_pred, _Y)
        losses = np.append(losses, (loss.item()))

        # find the class from the max values in each row
        _, predicted = torch.max(y_pred, dim = 1)

        # calculate and print accuracy
        total = predicted.size(0)
        correct = predicted.data.numpy() == _Y.data.numpy()
        accuracy = 100 * sum(correct)/total
        accuracies = np.append(accuracies,accuracy)

        optimizer.zero_grad()            # zero the gradients on each pass before the update
        loss.backward()                  # backpropagate the loss through the model
        optimizer.step()                 # update the gradients w.r.t the loss

    if e % 10 == 0:                      # validate every 10 epochs  
        print("Epoch: ", e)
        print("loss", sum(losses)/counter)
        print("accuracies", sum(accuracies)/counter)
    
        # find the validation error, here sequence length is 1, batch size is the length of validation data
        model.batch = X_validate.shape[0]
        model.seq_len = 1
        
        validate_y_pred = model(X_validate)
        validate_loss = loss_f(validate_y_pred, Y_validate)
        _, predicted = torch.max(validate_y_pred, dim = 1)
        total = predicted.size(0)
        correct = predicted.data.numpy() == Y_validate.data.numpy()
        accuracy = 100 * sum(correct)/total
        print("Validate", validate_loss.item(),accuracy)
        
        # terminate if validation loss is higher than previous two runs
        if validate_loss > max(previous_two_validation_error):
            print("terminated: at epoch ", e)
            break
        previous_two_validation_error[0] = previous_two_validation_error[1]
        previous_two_validation_error[-1] = validate_loss

Epoch:  0
loss 0.6935127013820713
accuracies 53.91949152542373
Validate 0.6833825707435608 56.82226211849192
Epoch:  10
loss 0.308817096054554
accuracies 86.06991525423729
Validate 0.2676123380661011 89.04847396768402
Epoch:  20
loss 0.2668918205772416
accuracies 88.29449152542372
Validate 0.26578378677368164 88.18073010173549
Epoch:  30
loss 0.22808208771176258
accuracies 90.3072033898305
Validate 0.2781328558921814 87.55236385397966
terminated: at epoch  30


In [319]:
"""
Step 5: Test the model
"""

# tune the parameters to fit the test data
model.batch = X_test.shape[0]
model.seq_len = 1
test_y_pred = model(X_test)
test_loss = loss_f(test_y_pred, Y_test)

_, predicted = torch.max(test_y_pred, 1)

total = predicted.size(0)
correct = sum(predicted.data.numpy() == Y_test.data.numpy())
print("Testing Loss: ", test_loss.item())
print("Testing Accuracy: %.2f %%" % (100 * correct / total))

Testing Loss:  0.4484092891216278
Testing Accuracy: 83.40 %
